In [50]:
import re
import numpy as np
import pandas as pd
from string import punctuation
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/glluch/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [59]:
allowed_punc = ['.', ',', '!', '@']
def remove_hash(text):
    return re.sub(r'#([^\s]+)', r'\1', text)

def remove_URL(text):
    return re.sub('((www\.[^r\s]+)|(https?://[^\rs]+))', 'URL', text)

def remove_punctuation(text):
    nopunc = [char for char in text if char in allowed_punc or char not in punctuation]
    return ''.join(nopunc)

def join_mention(text):
    return text.replace('@ ', '@')

def tokenize(text):
    return (word_tokenize(text))

def process(content): 
    text = content
    text = remove_hash(text)
    text = remove_URL(text)
    text = remove_punctuation(text)
    text = join_mention(text)
    
    return text

In [60]:
filepath = 'data/trumptweets.csv'

data = pd.read_csv(filepath)
# print(data.head())
for i in range(5): 
    print(process(data.iloc[i]['content']))

Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!
Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!
Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman URL
New Blog Post Celebrity Apprentice Finale and Lessons Learned Along the Way URL
My persona will never be that of a wallflower  I’d rather build walls than cling to them Donald J. Trump


In [61]:
sample = data.sample(n=10, random_state=42)

sample['content_clean'] = sample.apply(lambda row: process(row['content']), axis=1)
print(sample['content_clean'])



12208    Great ruling on wind farm in Scotland—very sma...
27415     MakeAmericaGreatAgain  Trump2016 LIFE CHANGIN...
26208     @ellenEspence Im not convinced that any candi...
26321    Dopey @GeorgeWill, a big proponent of the Iraq...
1992     The election result in France is very disappoi...
20920         @PerryGolf Top 100 Golf Courses @golfcom URL
30495    VOTER REGISTRATION DEADLINES TODAY. You can re...
23774    @absabella Mr.Trump speaks the TRUTH And Will ...
8552      @wvcarsong Okay, please say hello to her for me.
18481    Entrepreneurs Set the bar high. Do the best yo...
Name: content_clean, dtype: object
<class 'str'>


In [69]:
data['content_clean'] = data.apply(lambda row: process(row['content']), axis=1)
data.to_csv('data/trumptweets_clean.csv')

In [89]:
from keras.preprocessing.text import Tokenizer

num_words = 20000 
tokenizer = Tokenizer(num_words=num_words, filters='')

tokenizer.fit_on_texts(data['content_clean'])
encoded = tokenizer.texts_to_sequences(data['content_clean'])
flat_encoded = [enc for encoder in encoded for enc in encoder]
total_words = len(tokenizer.word_index) + 1
